In [1]:
import requests
from dataclasses import dataclass
from typing import List, Optional, Dict
from dotenv import load_dotenv
import os
from datetime import datetime

load_dotenv()


True

In [2]:
headers = {'Accept': 'application/json'}
response = requests.get(os.getenv('URL_PATH'), headers=headers)

In [5]:
@dataclass
class Seance:
    cinemaId: str
    cinemaName: str
    showtimes: List[datetime]

@dataclass
class Rating:
    score: Optional[float] = None
    count: Optional[int] = None

@dataclass
class Stats:
    userRating: Rating
    pressRating: Rating

@dataclass
class Seance:
    cinemaId: str
    cinemaName: str
    showtimes: List[datetime]

@dataclass
class Person:
    lastName: str
    firstName: str
    position: str
    pictureUrl: Optional[str] = None
    

@dataclass
class Movie:
    movieId: str
    title: str
    synopsis: str
    runtime: int
    posterUrl: Optional[str] = None
    genre: Optional[List[str]] = None
    languages: Optional[List[str]] = None
    stats: Optional[Stats] = None
    certificate: Optional[str] = None
    directors: Optional[List[Person]] = None
    actors: Optional[List[Person]] = None
    seances: Optional[List[Seance]] = None



In [6]:
def get_url_from_nested(data: dict, *keys) -> Optional[str]:
    """Extract URL from nested dictionary structure."""
    current = data
    for key in keys:
        if not isinstance(current, dict):
            return None
        current = current.get(key)
        if current is None:
            return None
    return current

def parse_person(data: dict, is_actor: bool = False) -> Optional[Person]:
    """Parse person data into Person object. Returns None if required fields are missing."""
    try:
        if is_actor:
            base = data.get('node', {}).get('actor', {})
        else:
            base = data.get('person', {})
        
        lastName = base.get('lastName')
        firstName = base.get('firstName')
        position = 'actor' if is_actor else data.get('position', {}).get('name')
        
        # Return None if any required field is missing
        if not all([lastName, firstName, position]):
            return None
            
        return Person(
            lastName=lastName,
            firstName=firstName,
            pictureUrl=get_url_from_nested(base, 'picture', 'url'),
            position=position
        )
    except Exception:
        return None

def parse_seances(element: dict) -> Optional[Dict[str, any]]:
    """Parse seances data into dictionary format."""
    try:
        cinema_id = 'test'
        cinema_name = 'test'
        showtimes = element.get('showtimes', {}).get('original', [])
        
        if not all([cinema_id, cinema_name, showtimes]):
            return None
            
        return {
            'cinemaId': cinema_id,
            'cinemaName': cinema_name,
            'showtimes': [ele['startsAt'] for ele in showtimes]
        }
    except Exception:
        return None

def parse_stats(stats_data: Optional[dict]) -> Optional[Stats]:
    """Parse stats data into Stats object."""
    if not stats_data:
        return None
    
    return Stats(
        userRating=Rating(
            score=stats_data.get('userRating', {}).get('score'),
            count=stats_data.get('userRating', {}).get('count')
        ),
        pressRating=Rating(
            score=stats_data.get('pressReview', {}).get('score'),
            count=stats_data.get('pressReview', {}).get('count')
        )
    )


def parse_seances(element: dict) -> Optional[Seance]:
    """Parse seances data into a Seance object."""
    try:
        cinema_id = 'test'
        cinema_name = 'test'
        showtimes = element.get('showtimes', {}).get('original', [])
        
        if not all([cinema_id, cinema_name, showtimes]):
            return None
            
        # Convert string timestamps to datetime objects
        parsed_showtimes = [
            datetime.fromisoformat(time['startsAt'].replace('Z', '+00:00'))
            for time in showtimes
        ]
        
        return Seance(
            cinemaId=cinema_id,
            cinemaName=cinema_name,
            showtimes=parsed_showtimes
        )
    except Exception:
        return None

def parse_movie_data(element: dict) -> Optional[Dict[str, Movie]]:
    """Parse movie data into Movie object. Returns None if required fields are missing."""
    try:
        movie = element.get('movie', {})
        movie_id = movie.get('id')
        title = movie.get('title')
        synopsis = movie.get('synopsis')
        runtime = movie.get('runtime')
        
        # Return None if any required field is missing
        if not all([movie_id, title, synopsis, runtime]):
            return None
            
        # Parse credits and actors, filtering out None values
        credits = movie.get('credits', [])
        parsed_credits = [p for p in (parse_person(ele) for ele in credits) if p is not None]
        
        actors = movie.get('cast', {}).get('edges', [])
        parsed_actors = [p for p in (parse_person(ele, is_actor=True) for ele in actors) if p is not None]
        
        parsed_seances = parse_seances(element)
        
        return {movie_id: Movie(
            movieId=movie_id,
            title=title,
            synopsis=synopsis,
            posterUrl=get_url_from_nested(movie, 'poster', 'url'),
            runtime=runtime,
            genre=[ele.get('translate') for ele in movie.get('genres', [])] if movie.get('genres') else None,
            languages=movie.get('languages'),
            stats=parse_stats(movie.get('stats')),
            certificate=get_url_from_nested(movie, 'releases', 0, 'certificate', 'label'),
            directors=parsed_credits if parsed_credits else None,
            actors=parsed_actors if parsed_actors else None,
            seances=parsed_seances
        )}
    except Exception:
        return None

# Usage
structured_output = []
for element in response.json()["results"]:
    movie_info = parse_movie_data(element)
    structured_output.append(movie_info)

In [7]:
structured_output

[{'TW92aWU6MzA0NDA5': Movie(movieId='TW92aWU6MzA0NDA5', title='Hiver à Sokcho', synopsis='A Sokcho, petite ville balnéaire de Corée du Sud, Soo-Ha, 23 ans, mène une vie routinière, entre ses visites à sa mère, marchande de poissons, et sa relation avec son petit ami, Jun-oh. L’arrivée d’un Français, Yan Kerrand, dans la petite pension dans laquelle Soo-Ha travaille, réveille en elle des questions sur sa propre identité et sur son père français dont elle ne sait presque rien. Tandis que l’hiver engourdit la ville, Soo-Ha et Yan Kerrand vont s’observer, se jauger, tenter de communiquer avec leurs propres moyens et tisser un lien fragile.', runtime='1h 45min', posterUrl='https://fr.web.img3.acsta.net/img/04/d7/04d71113c83535c44e8924e6e948275c.jpg', genre=['Drame'], languages=['FRENCH', 'KOREAN'], stats=Stats(userRating=Rating(score=3.81, count=316), pressRating=Rating(score=3.17, count=29)), certificate=None, directors=[Person(lastName='Kamura', firstName='Koya', position='DIRECTOR', pict